In [1]:
import numpy as np
import scipy.sparse as sparse
from tqdm import tqdm
import pandas as pd
import implicit
import pickle
import os

In [2]:
def make_encode_df(model_df, columns, path):
    encode_df = model_df[columns]
    encode_df = encode_df.drop_duplicates()
    encode_df.to_csv(path, index=False)


def make_df_for_model(counted_df, model_df_path):
    df = counted_df.copy()
    # конвртирование данных в коды категорий
    # данный формат необходим для обучения моделей , основанных на impicit
    df["user_id"] = df["user_id"].astype("category")
    df["product_id"] = df["product_id"].astype("category")
    df["user"] = df["user_id"].cat.codes
    df["product"] = df["product_id"].cat.codes

    df["count"] = df["count"].astype("category")
    df["count"] = df["count"].cat.codes
    df.to_csv(model_df_path, index=False)


def save_uniq_users(user_arr, uniq_users_path):
    uniq_users = user_arr.unique()
    np.save(uniq_users_path, uniq_users)


def read_data(products_path, transactions_path):
    """
    return products , transactions
    """
    products = pd.read_csv(products_path)
    transactions = pd.read_csv(transactions_path)
    return products, transactions


def make_counted_data(transactions, uniq_users, counted_df_path):
    """
    функция подсчитывания заказов
    """
    counter_all_users = {}
    for user in tqdm(uniq_users):
        user_transactions = transactions[
            transactions["user_id"] == user
        ]  # выбор всех транзакций для определенного пользователя - user
        list_of_orders = user_transactions[
            "product_id"
        ]  # получения списка продуктов из всех транзакций пользователя
        order_counter = (
            list_of_orders.value_counts()
        )  # подсчет кол-ва заказов для каждого продукта
        order_counter_dict = order_counter.to_dict()
        counter_all_users[user] = order_counter_dict
    counted_arr = []

    # разложения словаря в более удобный формат
    for user in counter_all_users.keys():
        for prod, cnt in counter_all_users[user].items():
            counted_arr.append([user, prod, cnt])
    make_counted_df = pd.DataFrame(
        counted_arr, columns=["user_id", "product_id", "count"]
    )

    # сохранение подсчитанного датасета
    make_counted_df.to_csv(counted_df_path, index=False)

In [3]:
class Data_worker:
    def __init__(
        self,
        products_path="data/products.csv",
        transactions_path="data/transactions.csv",
        uniq_users_path="data/uniq_users.npy",
        counted_df_path="data/user_product_counted.csv",
        model_df_path="data/model_df.csv",
        encode_products_path="data/encode_products.csv",
        encode_users_path="data/encode_users.csv",
    ):
        self.products_path = products_path
        self.transactions_path = transactions_path
        self.uniq_users_path = uniq_users_path
        self.counted_df_path = counted_df_path
        self.encode_products_path = encode_products_path
        self.encode_users_path = encode_users_path
        self.model_df_path = model_df_path
        self.products = None
        self.transactions = None
        self.uniq_users = None
        self.counted_df = None
        self.model_df = None
        self.encode_users = None
        self.encode_products = None

    def read_data(self):
        """
        Считывание transactions и products
        """
        self.products, self.transactions = read_data(
            self.products_path, self.transactions_path
        )
        print("Успешно считаны основные данные")

    def uniq_users_to_numpy(self):
        """
        Функция для выделения и сохранения уникальных пользователей
        """
        if self.transactions is None:
            raise ValueError("Данные не загружены")
        users = self.transactions["user_id"]
        save_uniq_users(users, self.uniq_users_path)
        print("Уникальные пользователи сохранены")

    def modef_df_to_csv(self):
        """
        Создание и сохранение датасета для обучения
        """
        if self.counted_df is None:
            raise ValueError("Данные не загружены")
        make_df_for_model(self.counted_df, self.model_df_path)
        print("Набор данных для обучения модели создан успешно")

    def counted_df_to_csv(self):
        """
        Создание и сохранение набора данных с подсчитанными заказами
        """
        if self.transactions is None:
            raise ValueError("Данные не загружены")
        if self.uniq_users is None:
            raise ValueError("Данные не загружены")
        make_counted_data(
            self.transactions, self.uniq_users, self.counted_df_path
        )
        print("Подсчитанный набор данных создан успешно")

    def make_products_encode(self):
        """
        Создание и сохранение энкодера для продуктов
        """
        if self.model_df is None:
            raise ValueError("Данные не загружены")
        make_encode_df(
            self.model_df, ["product_id", "product"], self.encode_products_path
        )
        print("Данные для декодирования продуктов созданы успешно")

    def make_users_encode(self):
        """
        Создание и сохранение энкодера для пользователей
        """
        if self.model_df is None:
            raise ValueError("Данные не загружены")
        make_encode_df(
            self.model_df, ["user_id", "user"], self.encode_users_path
        )
        print("Данные для декодирования пользователей созданы успешно")

    def read_counted_df(self):
        """
        Считывание датасета с подсчитанными заказами
        """
        if not os.path.exists(self.counted_df_path):
            raise OSError(
                "Неправильный путь к датасету с подсчитанными данными / файл не существует"
            )
        self.counted_df = pd.read_csv(self.counted_df_path)
        print("Успешно считаны подсчитанные данные")

    def read_uniq_users(self):
        """
        Считывание файла со списком уникальных пользователей
        """
        if not os.path.exists(self.uniq_users_path):
            raise OSError(
                "Неправильный путь к уникальным пользователям / файл не существует"
            )
        self.uniq_users = np.load(self.uniq_users_path)
        print("Уникальные пользователи считаны успешно")

    def read_model_df(self):
        """
        Считывание датасета для обучения модели
        """
        if not os.path.exists(self.model_df_path):
            raise OSError(
                "Неправильный путь к датасету для обучения / файл не существует"
            )
        self.model_df = pd.read_csv(self.model_df_path)
        print("Успешно считаны данные для обучения модели")

    def read_user_encode_df(self):
        """
        Считывание энкодера для пользователей
        """
        if not os.path.exists(self.encode_users_path):
            raise OSError(
                "Неправильный путь к енкодеру пользователей / файл не существует"
            )
        self.encode_users = pd.read_csv(self.encode_users_path)
        print("Успешно считаны данные для декодирования пользователей")

    def read_products_encode_df(self):
        """
        Считывание энкодера для продуктов
        """
        if not os.path.exists(self.encode_products_path):
            raise OSError("Неправильный путь к енкодеру продуктов / файл не существует")
        self.encode_products = pd.read_csv(self.encode_products_path)
        print("Успешно считаны данные для декодирования продуктов")

In [4]:
class Knn:
    def __init__(self, data, K=1):
        """
        Модель на основе алгоритма ближайших соседей
        K - количество соседей
        data - model_df
        """
        self.K = K
        self.data = data
        self.create_sprace_arrays()

    def create_sprace_arrays(self):
        """
        Создание разряженных матриц
        """
        self.sparse_item_user = sparse.csr_matrix(
            (
                self.data["count"].astype(float),
                (self.data["product"], self.data["user"]),
            )
        )
        self.sparse_user_item = sparse.csr_matrix(
            (
                self.data["count"].astype(float),
                (self.data["user"], self.data["product"]),
            )
        )
        print(self.sparse_item_user.shape)

    def fit(self):
        """
        Обучение модели
        """
        self.model = implicit.nearest_neighbours.CosineRecommender(K=self.K)
        self.model.fit(self.sparse_item_user)

    def predict_all(self, uniq_users, encode_products, encode_users, n=10):
        """
        предсказать продукты для всех user в массиве uniq_users
        n - количество предлагаемых продуктов
        """

        users_dict = {}
        for user in tqdm(uniq_users):
            decode_user = encode_users[encode_users["user_id"] == user]["user"].iloc[0]
            recommended = self.model.recommend(
                decode_user,
                self.sparse_user_item,
                filter_already_liked_items=False,
                recalculate_user=True,
                N=n,
            )
            user_rec_items = []
            for item in recommended:
                idx, score = item
                rec_item = (
                    encode_products["product_id"]
                    .loc[encode_products["product"] == idx]
                    .iloc[0]
                )
                user_rec_items.append(rec_item)
            users_dict[user] = user_rec_items
        return users_dict

    def predict_for_user(self, user, encode_products, encode_users, n=10):
        """
        предсказать продукты для конкретного пользователя
        n - количество предлагаемых продуктов
        """
        users_dict = {}
        decode_user = encode_users[encode_users["user_id"] == user]["user"].iloc[0]
        recommended = self.model.recommend(
            decode_user,
            self.sparse_user_item,
            N=n,
            filter_already_liked_items=False,
            recalculate_user=True,
        )
        user_rec_items = []
        for item in recommended:
            idx, score = item
            rec_item = (
                encode_products["product_id"]
                .loc[encode_products["product"] == idx]
                .iloc[0]
            )
            user_rec_items.append(rec_item)
        users_dict[user] = user_rec_items
        return users_dict

    def create_submission(
        self, users_dict, uniq_users, path="results/nearest_neighbours25.csv"
    ):
        """
        Создание структуры для отправки на Kaggle
        """
        for user in uniq_users:
            if user in users_dict.keys():
                users_dict[user] = " ".join(str(x) for x in users_dict[user])
        df_sub = pd.DataFrame(users_dict.items(), columns=["user_id", "product_id"])
        df_sub.to_csv(path, index=False)

    def save_model(self, weights_path):
        self.model.save(weights_path)

    def load_model(self, weights_path):
        self.model = implicit.nearest_neighbours.CosineRecommender(K=self.K)
        self.model.load(weights_path)

In [5]:
d = Data_worker()

d.read_counted_df()
d.modef_df_to_csv()
d.read_model_df()
d.read_products_encode_df()
d.read_user_encode_df()
d.read_uniq_users()
data = d.model_df
k = Knn(data, K=1)
k.fit()
k.save_model(weights_path="weights/knn_model")
k.load_model(weights_path="weights/knn_model")

Успешно считаны подсчитанные данные
Набор данных для обучения модели создан успешно
Успешно считаны данные для обучения модели
Успешно считаны данные для декодирования продуктов
Успешно считаны данные для декодирования пользователей
Уникальные пользователи считаны успешно
(49465, 100000)


/home/germa/miniconda3/envs/recsys/lib/python3.6/site-packages/implicit/nearest_neighbours.py:141: RuntimeWarning: invalid value encountered in true_divide
  X.data = X.data / sqrt(bincount(X.row, X.data ** 2))[X.row]
100%|██████████| 49465/49465 [00:02<00:00, 23973.49it/s]


In [12]:
from implicit import _nearest_neighbours

In [23]:
k.model.scorer

In [20]:
k.predict_for_user(1, d.encode_products, d.encode_users)

AttributeError: 'NoneType' object has no attribute 'recommend'

In [21]:
asd = k.predict_all(d.uniq_users, d.encode_products, d.encode_users)

  0%|          | 0/100000 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'recommend'